# Castro et al. (2019) Data Processing and Analysis

## Setup

In [1]:
import os
import sys
import glob
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import re
import pickle
from collections import Counter
from functools import reduce

import pandas as pd
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl

In [3]:
from cvtk.cvtk import TemporalFreqs, TiledTemporalFreqs
from cvtk.cov import stack_temporal_covariances
import cvtk.variant_files as vf
from cvtk.gintervals import GenomicIntervals
from cvtk.pca import FreqPCA
from cvtk.plots import rep_plot_pca, correction_diagnostic_plot
from cvtk.utils import integerize
from cvtk.utils import extract_empirical_nulls_diagonals, extract_temporal_cov_diagonals
from cvtk.cov import stack_replicate_covariances, stack_temporal_covs_by_group
from cvtk.variant_files import VCFFile

In [4]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
mpl.rcParams['figure.figsize'] = (8.0, 4.0)
mpl.rcParams['figure.dpi'] = 200

# Varianta Data Loading

### Load in VCF data

In [5]:
vcf = VCFFile('../data/castro_et_al_2019/beagle_genMap.all.impute.vcf.gz')

reading file '../data/castro_et_al_2019/beagle_genMap.all.impute.vcf.gz'...
file '../data/castro_et_al_2019/beagle_genMap.all.impute.vcf.gz' loaded.
total time to load VCF file: 9.510106468200684 mins.


Remove fixed sites — those that are not polymorphic in any samples / timepoints. These just needlessly shrink the covariance towards zero.

### Sample Data

The samples names to line/generation mapping was not in a simple text file, but I found the relevant information in the vcftools commands that were at the beginning of the file `Longshanks_F0F17.summary_stats.tar.gz`. From this I created `samples.txt`, which is read in and parsed below.

In [6]:
samples = pd.read_csv("../data/castro_et_al_2019/samples.txt", header=None, names = ('line', 'individual'))
sample_map = {k:v for k, v in zip(samples['individual'], samples['line'])}

subpop_indices = defaultdict(list)
for i, k in enumerate(vcf.samples):
    subpop_indices[sample_map[k.decode()]].append(i)

From this, we can map the `vcf.geno_mat` table to subpopulation counts. 

In [7]:
counts_mat = vcf.count_alleles_subpops(subpop_indices)

/home/vinceb/projects/cvtk/cvtk/variant_files.py:310: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  self.mat = np.stack(counts_mat.values())


In [8]:
vcf.subpops

dict_keys(['Ctrl_F17', 'LS1_F17', 'LS2_F17', 'Ctrl_F0', 'LS1_F0', 'LS2_F0'])

Now we count the number of diploids in each sample.

In [9]:
ndiploids = [Counter(sample_map.values())[k] for k in vcf.subpops]

In [10]:
def parse_samples(x):
    line, gen = x.split('_')
    return (line, gen[1:])

design = [parse_samples(x) for x in vcf.subpops]

In [11]:
freq_mat_all = vcf.calc_freqs()

In [12]:
print("number of loci: ", freq_mat_all.shape[1])

number of loci:  31944210


With the frequencies calculated, now we filter out all non-segregating sites.

In [13]:
vcf.remove_fixed()
freq_mat = vcf.calc_freqs()
print("number of loci: ", freq_mat.shape[1])
print("loci not segregating removed: ", freq_mat_all.shape[1] - freq_mat.shape[1])

number of loci:  8162172
loci not segregating removed:  23782038


In [14]:
gi = vcf.build_gintervals()

## Replicate Covariance Analysis

In [16]:
tile_width = 10e6
gi.infer_seqlens()
tiles = GenomicIntervals.from_tiles(gi.seqlens, width=tile_width)

In [17]:
d = TiledTemporalFreqs(tiles, freqs=freq_mat, depths=vcf.N, diploids=ndiploids, samples=design, gintervals=gi)

In [18]:
d.samples

[('Ctrl', '0'),
 ('Ctrl', '17'),
 ('LS1', '0'),
 ('LS1', '17'),
 ('LS2', '0'),
 ('LS2', '17')]

In [19]:
autosomes = list(set(gi.intervals.keys()) - set('chrX'))

In [20]:
covs_cis = d.bootstrap_cov(B=5000, keep_seqids=autosomes, average_replicates=False, progress_bar=True)

In [25]:
covs_cis

array([[[ 0.08216114, -0.0211423 , -0.01879697],
        [-0.0211423 ,  0.11589154,  0.01291927],
        [-0.01879697,  0.01291927,  0.13267256]],

       [[ 0.10591613, -0.00780514, -0.00404746],
        [-0.00780514,  0.14027914,  0.02612223],
        [-0.00404746,  0.02612223,  0.16853641]],

       [[ 0.1257028 ,  0.00491804,  0.01073867],
        [ 0.00491804,  0.16270104,  0.03954906],
        [ 0.01073867,  0.03954906,  0.19750535]]])

In [22]:
with open('../data/castro_et_al_2019/covs_bootstrap.npy', 'wb') as f:
    np.save(f, covs_cis)

### Bootstrap the Convergence Correlation

In [37]:
convergence_corr = d.bootstrap_convergence_corr(B=1000, progress_bar=True)

In [39]:
convergence_corr

array([[[-0.0178953 ]],

       [[ 0.03364797]],

       [[ 0.08345644]]])

In [34]:
convergence_corr

array([[[[-0.18739708, -0.17079451, -0.18739708,  0.09625715,
          -0.17079451,  0.09625715]],

        [[-0.13999533, -0.1325864 , -0.13999533,  0.09791242,
          -0.1325864 ,  0.09791242]],

        [[-0.1153194 , -0.10475765, -0.1153194 ,  0.07775817,
          -0.10475765,  0.07775817]]],


       [[[-0.0766596 , -0.03227367, -0.0766596 ,  0.24650968,
          -0.03227367,  0.24650968]],

        [[-0.05623302, -0.02637868, -0.05623302,  0.18661366,
          -0.02637868,  0.18661366]],

        [[-0.04656208, -0.0215006 , -0.04656208,  0.15206221,
          -0.0215006 ,  0.15206221]]],


       [[[ 0.05583696,  0.10032341,  0.05583696,  0.3731058 ,
           0.10032341,  0.3731058 ]],

        [[ 0.02927055,  0.08495252,  0.02927055,  0.27405476,
           0.08495252,  0.27405476]],

        [[ 0.02656788,  0.06815975,  0.02656788,  0.22601915,
           0.06815975,  0.22601915]]]])

## Analysis Excluding Chromosomes 5 and 10

In [26]:
autosomes_sans_chr5_and_chr10 = [chr for chr in autosomes if chr not in ('chr5', 'chr10')]

In [28]:
covs_sans_chr5_and_chr1_cis = d.bootstrap_cov(B=5000, keep_seqids=autosomes_sans_chr5_and_chr10, 
                                              average_replicates=False, progress_bar=True)

In [29]:
covs_sans_chr5_and_chr1_cis

array([[[ 0.08432771, -0.02343031, -0.02532361],
        [-0.02343031,  0.11319475,  0.0121667 ],
        [-0.02532361,  0.0121667 ,  0.1297967 ]],

       [[ 0.10991822, -0.00873688, -0.00959943],
        [-0.00873688,  0.13948149,  0.02558092],
        [-0.00959943,  0.02558092,  0.15266809]],

       [[ 0.13082629,  0.00537015,  0.00657944],
        [ 0.00537015,  0.1629076 ,  0.03876012],
        [ 0.00657944,  0.03876012,  0.17443489]]])

In [30]:
with open('../data/castro_et_al_2019/covs_sans_chr5_and_chr1_bootstrap.npy', 'wb') as f:
    np.save(f, covs_sans_chr5_and_chr1_cis)

# OLD STUFF

In [ ]:
design_orig = [('Ctrl', 0), ('Ctrl', 17), ('LS1', 0), ('LS1', 17), ('LS2', 0), ('LS2', 17)]

In [50]:
CACHED_FREQS = '../data/castro_et_al_2019/castro_et_al_2019_frequencies_new.pkl'
CACHED_POS = '../data/castro_et_al_2019/castro_et_al_2019_loci_new.pkl'

def process_sample_name(f):
    line, gen = os.path.basename(f).split('.')[-2].split('_')
    return line, int(gen[1:])

if not os.path.exists(CACHED_FREQS):
    print("processing frequency matrix from raw data...")
    frq_files = glob.glob('../data/castro_et_al_2019/*frq')
    all_data = dict()
    all_metadata = []

    first = True
    for frq_file in frq_files:
        m = re.match(r'beagle_genMap\.all\.impute\.(?P<line>\w+)_F(?P<gen>\w+).frq', os.path.basename(frq_file))
        metadata = m.groupdict()
        all_metadata.append(metadata)
        da = pd.read_csv(frq_file, delimiter='\t', index_col=False)
        da.columns = ['chrom', 'pos', 'nalleles', 'nchr', 'freq']
        # assert the data has the exact same loci in all cases
        if first:
            loci = set([(chrom, pos) for chrom, pos in zip(da['chrom'], da['pos'])])
        else:
            assert(loci == set([(chrom, pos) for chrom, pos in zip(da['chrom'], da['pos'])]))
        all_data[(metadata['line'], int(metadata['gen']))] = da
    
    # design — from file names
    
    design_orig = [process_sample_name(f) for f in frq_files]
    # combine all frequencies into a matrix
    total_freq_mat = np.array([all_data[key]['freq'].values for key in design_orig])
    
    # filter out non-segregating sites
    fixed_sites = np.all((total_freq_mat == 0) | (total_freq_mat == 1), axis=0)
    freq_mat_orig = total_freq_mat[:, ~fixed_sites]
    seg_indices = set(list(np.argwhere(~fixed_sites).ravel()))
    
    # cache the frequency matrix
    print("saving frequency matrix cached pickle file...")
    with open(CACHED_FREQS, 'wb') as f:
        pickle.dump(freq_mat, f)
        
    print("rebuilding loci GenomicIntervals...")
    gi_orig = GenomicIntervals()
    for i, row in enumerate(all_data[('Ctrl', 0)].itertuples(index=False)):
        if i not in seg_indices:
            # We only keep segregating sites, which are filtered out once the frequency
            # matrix is parsed. We don't include loci that aren't in the freq matrix.
            continue
        chrom, pos = row[0:2]
        gi_orig.append(chrom, int(pos))
    print("caching loci GenomicIntervals...")
    gi_orig.dump(CACHED_POS)
else:
    # load the existing cached matrix
    print("loading frequency matrix from cached pickle file...")
    with open(CACHED_FREQS, 'rb') as f:
        freq_mat_orig = pickle.load(f)

saving frequency matrix cached pickle file...
rebuilding loci GenomicIntervals...
caching loci GenomicIntervals...


In [80]:
# try to load loci file
if not os.path.exists(CACHED_POS):
    raise ValueError("cached loci file not found — regenerate frequency matrix and loci GenomicIntervals object.")
else:
    gi_orig = GenomicIntervals.load(CACHED_POS)

In [53]:
autosomes = list(set(gi_orig.intervals.keys()) - set('chrX'))

In [105]:
tile_width = 1e6
gi_orig.infer_seqlens()
tiles = GenomicIntervals.from_tiles(gi_orig.seqlens, width=tile_width)

In [106]:
d_orig = TiledTemporalFreqs(tiles, freqs=freq_mat_orig, samples=design_orig, gintervals=gi_orig)

In [107]:
d_orig.samples

[('Ctrl', 0), ('Ctrl', 17), ('LS1', 0), ('LS1', 17), ('LS2', 0), ('LS2', 17)]

In [108]:
cis = d_orig.bootstrap_covs(bias_correction=True, B=1000)

In [109]:
cis

array([[[ 0.15706537, -0.01311851, -0.02019287],
        [-0.01311851,  0.1931862 ,  0.01537248],
        [-0.02019287,  0.01537248,  0.17993647]],

       [[ 0.19202768, -0.00726231,  0.00861766],
        [-0.00726231,  0.21124166,  0.02385372],
        [ 0.00861766,  0.02385372,  0.26658358]],

       [[ 0.21883855, -0.0015765 ,  0.02425131],
        [-0.0015765 ,  0.22675749,  0.03211054],
        [ 0.02425131,  0.03211054,  0.31294379]]])

In [142]:
cis = d_orig.bootstrap_covs(bias_correction=True, B=1000)

In [143]:
cis

array([[[ 0.15223608, -0.01292397, -0.01651098],
        [-0.01292397,  0.19240113,  0.01592663],
        [-0.01651098,  0.01592663,  0.19384668]],

       [[ 0.19238756, -0.00699839,  0.00872394],
        [-0.00699839,  0.21070029,  0.02368889],
        [ 0.00872394,  0.02368889,  0.26627927]],

       [[ 0.2195354 , -0.00119212,  0.02399211],
        [-0.00119212,  0.22657348,  0.03065855],
        [ 0.02399211,  0.03065855,  0.31239055]]])

In [144]:
d.bootstrap_covs(bias_correction=False, B=1000)

array([[[ 1.64739558e-01, -6.98373032e-03, -4.17432904e-03],
        [-6.98373032e-03,  1.59469167e-01,  1.58613301e-02],
        [-4.17432904e-03,  1.58613301e-02,  1.28936642e-01]],

       [[ 1.84694156e-01, -3.19842661e-03, -1.07405274e-03],
        [-3.19842661e-03,  2.67469243e-01,  3.12190640e-02],
        [-1.07405274e-03,  3.12190640e-02,  3.80804354e-01]],

       [[ 1.98588563e-01,  1.52196594e-04,  1.95032828e-03],
        [ 1.52196594e-04,  3.27814523e-01,  4.04793145e-02],
        [ 1.95032828e-03,  4.04793145e-02,  5.16389266e-01]]])

In [145]:
d.bootstrap_covs(bias_correction=True, B=1000)

array([[[ 1.23933927e-01, -7.19440902e-03, -4.52130752e-03],
        [-7.19440902e-03,  1.28829188e-01,  1.57922725e-02],
        [-4.52130752e-03,  1.57922725e-02,  1.05781346e-01]],

       [[ 1.40663111e-01, -3.16359676e-03, -1.20428299e-03],
        [-3.16359676e-03,  2.20916818e-01,  3.14401267e-02],
        [-1.20428299e-03,  3.14401267e-02,  3.35132087e-01]],

       [[ 1.51555866e-01,  3.96539427e-04,  1.72468667e-03],
        [ 3.96539427e-04,  2.72891812e-01,  4.06234834e-02],
        [ 1.72468667e-03,  4.06234834e-02,  4.64063119e-01]]])

In [146]:
d.samples

[('Ctrl', '0'),
 ('Ctrl', '17'),
 ('LS1', '0'),
 ('LS1', '17'),
 ('LS2', '0'),
 ('LS2', '17')]

In [147]:
d_orig.samples

[('Ctrl', 0), ('Ctrl', 17), ('LS1', 0), ('LS1', 17), ('LS2', 0), ('LS2', 17)]

In [153]:
da = TiledTemporalFreqs(tiles, freqs=freq_mat, depths=vcf.N, diploids=ndiploids, samples=design, gintervals=gi)

In [154]:
da.bootstrap_covs(bias_correction=True, B=1000)

array([[[ 0.11101553, -0.01276927, -0.01793371],
        [-0.01276927,  0.15460198,  0.01523274],
        [-0.01793371,  0.01523274,  0.15884918]],

       [[ 0.1428758 , -0.00693652,  0.00862564],
        [-0.00693652,  0.17083115,  0.02340974],
        [ 0.00862564,  0.02340974,  0.21297644]],

       [[ 0.16408569, -0.00125207,  0.02385345],
        [-0.00125207,  0.18449644,  0.03165025],
        [ 0.02385345,  0.03165025,  0.2451751 ]]])

In [156]:
db = TiledTemporalFreqs(tiles, freqs=freq_mat, samples=design, gintervals=gi)
db.bootstrap_covs(bias_correction=False, B=1000)

array([[[ 0.15658179, -0.01329954, -0.01855293],
        [-0.01329954,  0.19376705,  0.01506235],
        [-0.01855293,  0.01506235,  0.17660543]],

       [[ 0.19224338, -0.00722405,  0.0089552 ],
        [-0.00722405,  0.21088039,  0.02386709],
        [ 0.0089552 ,  0.02386709,  0.26673605]],

       [[ 0.21846683, -0.0010027 ,  0.02436376],
        [-0.0010027 ,  0.22454177,  0.03155517],
        [ 0.02436376,  0.03155517,  0.31371142]]])